In [1]:
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import sqlite3

In [2]:
#connect to database
path = "model/db/HeartDiseaseByCounty.db"
cnx = sqlite3.connect(path)

In [3]:
#Display tables from provisional database
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", cnx)

,name
0,State
1,Heart_Disease_Mortality_State_Counties
2,Features_State_Counties
3,Population_State_Counties


In [4]:
#Read all tables from provisional database into DataFrames
df_features = pd.read_sql_query("SELECT * FROM Features_State_Counties", cnx)
df_features.head()

,State_Name,County,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,...,Social_Association_Rate,Violent_Crime_Rate,Polution_Average_Daily_PM2.5,Presence_of_Water_Violation,Percent_Severe_Housing_Problems,Percent_Drive_Alone_to_Work,Percent_Long_Commute,Percent_Adults_with_Diabetes,Percent_Limited_Access_to_Healthy_Foods,Median_Household_Income
0,Alabama,Autauga,21,4.7,4.7,18,33,7.2,35,69,...,12.1,272,11.7,0,15,87,40,11,12,59338
1,Alabama,Baldwin,18,4.2,4.3,17,31,8.0,27,74,...,10.2,204,10.3,0,14,84,42,11,5,57588
2,Alabama,Barbour,30,5.4,5.2,22,42,5.6,24,53,...,7.5,414,11.5,0,15,83,32,18,11,34382
3,Alabama,Bibb,19,4.6,4.6,19,38,7.8,34,16,...,8.4,89,11.2,0,10,85,50,15,3,46064
4,Alabama,Blount,22,4.9,4.9,19,34,8.4,30,16,...,8.4,483,11.7,0,11,86,59,17,3,50412


In [5]:
df_state = pd.read_sql_query("SELECT * FROM State", cnx)
df_state.head()

,Name,Code
0,Alaska,AK
1,Alabama,AL
2,Arkansas,AR
3,Arizona,AZ
4,California,CA


In [6]:
df_pop = pd.read_sql_query("SELECT * FROM Population_state_counties", cnx)
df_pop.head()

,State_Code,County,PopEst
0,AK,Aleutians East,"3,337"
1,AK,Aleutians West,"5,634"
2,AK,Anchorage,"288,000"
3,AK,Bethel,"18,386"
4,AK,Bristol Bay,836


In [7]:
df_mortality = pd.read_sql_query("SELECT * FROM Heart_disease_mortality_state_counties", cnx)
df_mortality.head()

,State_Code,County,Rate,Level
0,AK,Aleutians East,165.000000,1
1,AK,Aleutians West,261.800000,1
2,AK,Anchorage,261.733333,1
3,AK,Bethel,321.322222,2
4,AK,Bristol Bay,0.000000,1


In [8]:
#Pre-process data - standardize and convert all State names into State codes
statename = df_state["Name"].values.tolist()


In [9]:
symbol = df_state["Code"].values.tolist()


In [10]:
#Pre-process data - transform State_Name into codes for later merging 
df_features.replace(to_replace = statename, value = symbol, inplace = True)
df_features.tail()

,State_Name,County,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,...,Social_Association_Rate,Violent_Crime_Rate,Polution_Average_Daily_PM2.5,Presence_of_Water_Violation,Percent_Severe_Housing_Problems,Percent_Drive_Alone_to_Work,Percent_Long_Commute,Percent_Adults_with_Diabetes,Percent_Limited_Access_to_Healthy_Foods,Median_Household_Income
3137,WY,Sweetwater,15,3.4,3.6,18,30,7.7,25,90,...,10.3,300,5.1,1,10,76,18,9,11,73315
3138,WY,Teton,12,3.0,3.2,15,12,8.2,12,100,...,16.3,0,4.9,1,17,67,14,2,7,99087
3139,WY,Uinta,16,3.6,3.7,17,36,7.4,27,84,...,2.9,71,5.9,1,11,77,19,11,10,63401
3140,WY,Washakie,16,3.6,3.7,17,29,8.3,28,83,...,16.1,78,4.8,0,10,79,7,12,4,55190
3141,WY,Weston,14,3.5,3.7,17,33,7.9,27,63,...,13.0,157,4.1,0,14,74,24,9,4,54319


In [11]:
#Merge first two DataFrames together
df_combined1 = df_features.merge(df_mortality, left_on = ["State_Name","County"], right_on = ["State_Code","County"])
df_combined1.head()

,State_Name,County,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,...,Presence_of_Water_Violation,Percent_Severe_Housing_Problems,Percent_Drive_Alone_to_Work,Percent_Long_Commute,Percent_Adults_with_Diabetes,Percent_Limited_Access_to_Healthy_Foods,Median_Household_Income,State_Code,Rate,Level
0,AL,Autauga,21,4.7,4.7,18,33,7.2,35,69,...,0,15,87,40,11,12,59338,AL,422.022222,4
1,AL,Baldwin,18,4.2,4.3,17,31,8.0,27,74,...,0,14,84,42,11,5,57588,AL,321.570588,2
2,AL,Barbour,30,5.4,5.2,22,42,5.6,24,53,...,0,15,83,32,18,11,34382,AL,461.144444,4
3,AL,Bibb,19,4.6,4.6,19,38,7.8,34,16,...,0,10,85,50,15,3,46064,AL,393.036364,3
4,AL,Blount,22,4.9,4.9,19,34,8.4,30,16,...,0,11,86,59,17,3,50412,AL,387.481818,3


In [12]:
df_combined1.shape

(3013, 34)

In [13]:
#Pre-process data - standardize County name and convert PopEst into integers
df_pop['County'] = df_pop["County"].str.replace(' County', "")
df_pop["PopEst"] = pd.to_numeric(df_pop["PopEst"].str.replace(',',''))
df_pop.head()

,State_Code,County,PopEst
0,AK,Aleutians East,3337.0
1,AK,Aleutians West,5634.0
2,AK,Anchorage,288000.0
3,AK,Bethel,18386.0
4,AK,Bristol Bay,NaN


In [14]:
df_pop.dropna(inplace = True)
df_pop.isnull().sum

<bound method DataFrame.sum of       State_Code  County  PopEst
0          False   False   False
1          False   False   False
2          False   False   False
3          False   False   False
5          False   False   False
...          ...     ...     ...
3137       False   False   False
3138       False   False   False
3139       False   False   False
3140       False   False   False
3141       False   False   False

[3105 rows x 3 columns]>

In [15]:
#Merge all DataFrames together 

df_combined = df_combined1.merge(df_pop, how = "inner", left_on = ["State_Name","County"], right_on = ["State_Code","County"])
df_combined.head()

,State_Name,County,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,...,Percent_Drive_Alone_to_Work,Percent_Long_Commute,Percent_Adults_with_Diabetes,Percent_Limited_Access_to_Healthy_Foods,Median_Household_Income,State_Code_x,Rate,Level,State_Code_y,PopEst
0,AL,Autauga,21,4.7,4.7,18,33,7.2,35,69,...,87,40,11,12,59338,AL,422.022222,4,AL,55869.0
1,AL,Baldwin,18,4.2,4.3,17,31,8.0,27,74,...,84,42,11,5,57588,AL,321.570588,2,AL,223234.0
2,AL,Barbour,30,5.4,5.2,22,42,5.6,24,53,...,83,32,18,11,34382,AL,461.144444,4,AL,24686.0
3,AL,Bibb,19,4.6,4.6,19,38,7.8,34,16,...,85,50,15,3,46064,AL,393.036364,3,AL,22394.0
4,AL,Blount,22,4.9,4.9,19,34,8.4,30,16,...,86,59,17,3,50412,AL,387.481818,3,AL,57826.0


In [16]:
# Drop State and county columns as they are identifier, and remove Level (classification target),
#Remove "Rate" (from which we derive mortality classes)

X = df_combined.copy()
X = X.drop([ "State_Name","County", "Rate","Level", "State_Code_x", "State_Code_y"], 1)
X.head()

,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,Percent_Excessive_Drinking,Percent_Uninsured,...,Violent_Crime_Rate,Polution_Average_Daily_PM2.5,Presence_of_Water_Violation,Percent_Severe_Housing_Problems,Percent_Drive_Alone_to_Work,Percent_Long_Commute,Percent_Adults_with_Diabetes,Percent_Limited_Access_to_Healthy_Foods,Median_Household_Income,PopEst
0,21,4.7,4.7,18,33,7.2,35,69,15,9,...,272,11.7,0,15,87,40,11,12,59338,55869.0
1,18,4.2,4.3,17,31,8.0,27,74,18,11,...,204,10.3,0,14,84,42,11,5,57588,223234.0
2,30,5.4,5.2,22,42,5.6,24,53,13,12,...,414,11.5,0,15,83,32,18,11,34382,24686.0
3,19,4.6,4.6,19,38,7.8,34,16,16,10,...,89,11.2,0,10,85,50,15,3,46064,22394.0
4,22,4.9,4.9,19,34,8.4,30,16,14,13,...,483,11.7,0,11,86,59,17,3,50412,57826.0


In [17]:
#Target: Classification of mortality risks 
y = df_combined["Level"].ravel()
y[:5]

array([4, 2, 4, 3, 3])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [19]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [21]:
rf_model = rf_model.fit(X_train_scaled, y_train)

In [22]:
predictions = rf_model.predict(X_test_scaled)

In [23]:
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 1", "Actual 2", "Actual 3", "Actual 4"], columns=["Predicted 1", "Predicted 2", "Predicted 3", "Predicted 4"])

cm_df

,Predicted 1,Predicted 2,Predicted 3,Predicted 4
Actual 1,95,73,4,2
Actual 2,30,141,24,13
Actual 3,4,50,45,59
Actual 4,3,19,21,151


In [24]:
acc_score = accuracy_score(y_test, predictions)
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

,Predicted 1,Predicted 2,Predicted 3,Predicted 4
Actual 1,95,73,4,2
Actual 2,30,141,24,13
Actual 3,4,50,45,59
Actual 4,3,19,21,151


Accuracy Score : 0.5885558583106267
Classification Report
              precision    recall  f1-score   support

           1       0.72      0.55      0.62       174
           2       0.50      0.68      0.57       208
           3       0.48      0.28      0.36       158
           4       0.67      0.78      0.72       194

    accuracy                           0.59       734
   macro avg       0.59      0.57      0.57       734
weighted avg       0.59      0.59      0.58       734



In [25]:
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.07109090662902069, 'Median_Household_Income'),
 (0.05103514411721596, 'Percent_Smokers'),
 (0.049437951310902103, 'PopEst'),
 (0.0456427849850421, 'Polution_Average_Daily_PM2.5'),
 (0.0453747050084182, 'Average_Number_of_Physically_Unhealthy_Days'),
 (0.04378460344741857, 'Percent_Physically_Inactive'),
 (0.04119067737028978, 'Percent_Fair_or_Poor_Health'),
 (0.040210525085504226, 'Average_Number_of_Mentally_Unhealthy_Days'),
 (0.03731453784777011, 'Preventable_Hospitalization_Rate'),
 (0.03422261487561973, 'Percent_Some_College'),
 (0.03300984125621178, 'Social_Association_Rate'),
 (0.03282691648202782, 'Percent_With_Access_to_Exercise_Opportunities'),
 (0.030010709951447043, 'Percent_Unemployed'),
 (0.029792205254581188, 'Violent_Crime_Rate'),
 (0.029710918201429404, 'Primary_Care_Physicians_Rate'),
 (0.029701657221862958, 'Mental_Health_Provider_Rate'),
 (0.029513560341343156, 'Percent_Drive_Alone_to_Work'),
 (0.0289093878332154, 'Percent_Vaccinated'),
 (0.028145913240944034, 'D